In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score

In [2]:
def scores(df, df_name="labeled_data"):
    """
    only for the dataset
    """
    

    y_true = df[['counted_num']]
    y_pred = df[['detected_num']]

    label(df, "counted_num", "label_counted")
    label(df, "detected_num", "label_detected")
    issame(df)

    y_true_label = df[['label_counted']]
    y_pred_label = df[['label_detected']]

    accuracy_label = accuracy_score(y_true_label, y_pred_label)
    precision_label = precision_score(y_true_label, y_pred_label, average="micro")
    recall_label = recall_score(y_true_label, y_pred_label, average="micro")
    f1_label = f1_score(y_true_label, y_pred_label, average="micro")

    r2 = r2_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="micro")
    recall = recall_score(y_true, y_pred, average="micro")
    f1 = f1_score(y_true, y_pred, average="micro")

    accuracy_same = len(df[df['label_same'] == 'same']) / len(df)

    scores = []
    score = [
        df_name, accuracy, f1, precision, recall, r2, 
        accuracy_label, f1_label, precision_label, recall_label, 
        accuracy_same
    ]
    scores.append(score)
    scores = pd.DataFrame(
        np.array(scores),
        columns=[
            'df_name', 'accuracy', 'f1_score', 'precision_score',
            'recall_score', 'r2_score', 
            'accuracy_with_range', 'f1_with_range','precision_with_range', 'recall_with_range', 
            'accuracy_with_bool'
        ])
    display(scores.head())

    display(df.head())
    df.to_csv(path_or_buf=os.path.join(os.getcwd(), "%s.csv"%df_name))
    return scores

In [3]:
def label(df,target_fearure,label_name):
    df_label = df[target_fearure]
    label = pd.Series([])
    p1 = df_label.quantile(0.2)
    p2 = df_label.quantile(0.4)
    p3 = df_label.quantile(0.6)
    p4 = df_label.quantile(0.8)
    p5 = df_label.quantile(1)
    for j in range(len(df)):
#         print(len(df))
        if df_label.iloc[j] <= p1:
            label[j] = "<0.2"
        elif df_label.iloc[j] > p1 and df_label.iloc[j] <= p2:
            label[j] = "0.2 - 0.4"
        elif df_label.iloc[j] > p2 and df_label.iloc[j] <= p3:
            label[j] = "0.4-0.6"
        elif df_label.iloc[j] > p3 and df_label.iloc[j] <= p4:
            label[j] = "0.6-0.8"
        elif df_label.iloc[j] > p4 and df_label.iloc[j] <= p5:
            label[j] = "0.8-1"
    df.insert(df.shape[1],label_name,label)
    return df

In [4]:
def issame(df):
    """
    if the difference between the truth and predicted smaller than 5, we assume they are the same.
    """
    same = pd.Series([])
    y_true = df[['counted_num']]
    median = np.median(y_true.values)
    
    df1 = df[(df['counted_num'] >= median)]
    y_true1 = df1[['counted_num']]
    y_pred1 = df1[['detected_num']]
    
    df2 = df[(df['counted_num'] < median)]
    y_true2 = df2[['counted_num']]
    y_pred2 = df2[['detected_num']]
    
    std1 = np.std(np.abs(y_true1.values - y_pred1.values))
    std2 = np.std(np.abs(y_true2.values - y_pred2.values))
    print (median, std1, std2)
    for j in range(len(df)):
        if df.iloc[j].loc['counted_num'] >= median:
            if np.abs(df.iloc[j].loc['counted_num'] - df.iloc[j].loc['detected_num']) <= std1:
                same[j] = "same"
            else:
                same[j] = "not same"
        else:
            if np.abs(df.iloc[j].loc['counted_num'] - df.iloc[j].loc['detected_num']) <= std2:
                same[j] = "same"
            else:
                same[j] = "not same"

    df.insert(df.shape[1], "label_same", same) 
    return df

In [5]:
df = pd.read_csv(".\Dublin_night.csv")
s1 = scores(df,"labeled_dublin_night",)

23.0 5.163166085229937 2.2138296381308717


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,labeled_dublin_night,0.24,0.24,0.24,0.24,0.0495270931370263,0.52,0.52,0.52,0.52,0.52


,Unnamed: 0,image_name,detected_num,time,counted_num,label_counted,label_detected,label_same
0,0,dublin_night0.png,19,Thu 2018-12-27 21:15:12,24,0.4-0.6,0.4-0.6,same
1,1,dublin_night1.png,18,Thu 2018-12-27 21:18:23,25,0.4-0.6,0.4-0.6,not same
2,2,dublin_night2.png,21,Thu 2018-12-27 21:21:31,40,0.8-1,0.6-0.8,not same
3,3,dublin_night3.png,23,Thu 2018-12-27 21:24:39,37,0.8-1,0.8-1,not same
4,4,dublin_night4.png,29,Thu 2018-12-27 21:27:47,40,0.8-1,0.8-1,not same


In [6]:
df = pd.read_csv(".\Dublin_day.csv")
s2 = scores(df,"labeled_dublin_day")

15.5 9.857869952479591 2.0489997559785116


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,labeled_dublin_day,0.12,0.12,0.12,0.12,0.4813387789658524,0.58,0.58,0.58,0.58,0.72


,Unnamed: 0,image_name,detected_num,time,counted_num,label_counted,label_detected,label_same
0,0,dublin_day0.png,3,Fri 2018-12-28 10:15:08,4,<0.2,<0.2,same
1,1,dublin_day1.png,20,Fri 2018-12-28 10:18:18,23,0.6-0.8,0.6-0.8,same
2,2,dublin_day2.png,3,Fri 2018-12-28 10:21:23,6,<0.2,<0.2,not same
3,3,dublin_day3.png,1,Fri 2018-12-28 10:24:29,2,<0.2,<0.2,same
4,4,dublin_day4.png,3,Fri 2018-12-28 10:27:35,2,<0.2,<0.2,same


In [7]:
s = s1.append(s2)

In [8]:
s

,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,labeled_dublin_night,0.24,0.24,0.24,0.24,0.0495270931370263,0.52,0.52,0.52,0.52,0.52
0,labeled_dublin_day,0.12,0.12,0.12,0.12,0.4813387789658524,0.58,0.58,0.58,0.58,0.72


In [9]:
s.to_csv(path_or_buf=os.path.join(os.getcwd(), "dublin_scores.csv"))

In [23]:
df = pd.read_csv(".\Algos_dublin_night.csv")
df_resnet = df[df['algo']=="resnet"]
df_resnet = df_resnet.reset_index(drop=True)
s3 = scores(df_resnet,'lableled_resnet_dublin_night')

25.0 3.9350190596742984 2.996596709057576


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_resnet_dublin_night,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.6969771806776647,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.6


,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,0,resnet_dublin_night3.png,resnet,29,37,0.8-1,0.8-1,not same
1,1,resnet_dublin_night46.png,resnet,21,14,<0.2,0.2 - 0.4,not same
2,2,resnet_dublin_night18.png,resnet,19,18,0.2 - 0.4,0.2 - 0.4,same
3,3,resnet_dublin_night31.png,resnet,22,35,0.6-0.8,0.4-0.6,not same
4,4,resnet_dublin_night2.png,resnet,39,40,0.8-1,0.8-1,same


In [24]:
df_yolo = df[df['algo']=="yolov3"]
df_yolo = df_yolo.reset_index()
s4 = scores(df_yolo,'lableled_yolov3_dublin_night')

25.0 5.946374946133148 1.8405855323893037


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_yolov3_dublin_night,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,-0.398794823668873,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.2


,index,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,15,15,yolov3_dublin_night3.png,yolov3,20,37,0.8-1,0.6-0.8,not same
1,16,16,yolov3_dublin_night46.png,yolov3,18,14,<0.2,0.4-0.6,not same
2,17,17,yolov3_dublin_night18.png,yolov3,14,18,0.2 - 0.4,<0.2,not same
3,18,18,yolov3_dublin_night31.png,yolov3,15,35,0.6-0.8,0.2 - 0.4,not same
4,19,19,yolov3_dublin_night2.png,yolov3,20,40,0.8-1,0.6-0.8,not same


In [25]:
df_yolot = df[df['algo']=="yolo_tiny"]
df_yolot = df_yolot.reset_index()
s5 = scores(df_yolot,'lableled_yolot_dublin_night')

25.0 4.795831523312719 3.2450904833144416


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_yolot_dublin_night,0.0,0.0,0.0,0.0,-1.5167934406796406,0.4,0.4000000000000001,0.4,0.4,0.06666666666666667


,index,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,30,30,yolo_tiny_dublin_night3.png,yolo_tiny,19,37,0.8-1,0.8-1,not same
1,31,31,yolo_tiny_dublin_night46.png,yolo_tiny,11,14,<0.2,0.2 - 0.4,same
2,32,32,yolo_tiny_dublin_night18.png,yolo_tiny,5,18,0.2 - 0.4,<0.2,not same
3,33,33,yolo_tiny_dublin_night31.png,yolo_tiny,15,35,0.6-0.8,0.6-0.8,not same
4,34,34,yolo_tiny_dublin_night2.png,yolo_tiny,17,40,0.8-1,0.8-1,not same


In [26]:
sa = s3.append(s4).append(s5)

In [27]:
sa

,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_resnet_dublin_night,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.6969771806776647,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.6
0,lableled_yolov3_dublin_night,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,-0.398794823668873,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.2
0,lableled_yolot_dublin_night,0.0,0.0,0.0,0.0,-1.5167934406796406,0.4,0.4000000000000001,0.4,0.4,0.06666666666666667


In [15]:
s.to_csv(path_or_buf=os.path.join(os.getcwd(), "algos_dublin_night_scores.csv"))

In [28]:
df = pd.read_csv(".\Algos_dublin_day.csv")
df_resnet = df[df['algo']=="resnet"]
df_resnet = df_resnet.reset_index(drop=True)
s6 = scores(df_resnet,'lableled_resnet_dublin_day')

7.0 6.142831187652807 0.7284313590846835


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_resnet_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.8567149895438273,1.0,1.0,1.0,1.0,0.6666666666666666


,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,0,resnet_dublin_day41.png,resnet,25,23,0.6-0.8,0.6-0.8,same
1,1,resnet_dublin_day34.png,resnet,13,19,0.6-0.8,0.6-0.8,same
2,2,resnet_dublin_day10.png,resnet,1,2,<0.2,<0.2,not same
3,3,resnet_dublin_day2.png,resnet,8,6,0.4-0.6,0.4-0.6,not same
4,4,resnet_dublin_day5.png,resnet,4,4,0.2 - 0.4,0.2 - 0.4,same


In [29]:
df_yolo = df[df['algo']=="yolov3"]
df_yolo = df_yolo.reset_index()
s7 = scores(df_yolo,'lableled_yolov3_dublin_day')

7.0 11.129212685540699 0.4948716593053935


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_yolov3_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.47231772587105236,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.6


,index,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,15,15,yolov3_dublin_day41.png,yolov3,16,23,0.6-0.8,0.6-0.8,same
1,16,16,yolov3_dublin_day34.png,yolov3,12,19,0.6-0.8,0.6-0.8,same
2,17,17,yolov3_dublin_day10.png,yolov3,3,2,<0.2,<0.2,not same
3,18,18,yolov3_dublin_day2.png,yolov3,5,6,0.4-0.6,0.4-0.6,not same
4,19,19,yolov3_dublin_day5.png,yolov3,4,4,0.2 - 0.4,0.2 - 0.4,same


In [30]:
df_yolot = df[df['algo']=="yolo_tiny"]
df_yolot = df_yolot.reset_index()
s8 = scores(df_yolot,'lableled_yolot_dublin_day')

7.0 12.30853362509117 0.9258200997725514


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_yolot_dublin_day,0.0,0.0,0.0,0.0,0.22163541629745154,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.3333333333333333


,index,Unnamed: 0,image_name,algo,detected_num,counted_num,label_counted,label_detected,label_same
0,30,30,yolo_tiny_dublin_day41.png,yolo_tiny,15,23,0.6-0.8,0.8-1,same
1,31,31,yolo_tiny_dublin_day34.png,yolo_tiny,12,19,0.6-0.8,0.6-0.8,same
2,32,32,yolo_tiny_dublin_day10.png,yolo_tiny,1,2,<0.2,<0.2,not same
3,33,33,yolo_tiny_dublin_day2.png,yolo_tiny,3,6,0.4-0.6,0.4-0.6,not same
4,34,34,yolo_tiny_dublin_day5.png,yolo_tiny,1,4,0.2 - 0.4,<0.2,not same


In [32]:
sb = s6.append(s7).append(s8)
display(sb)


,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_resnet_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.8567149895438273,1.0,1.0,1.0,1.0,0.6666666666666666
0,lableled_yolov3_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.47231772587105236,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.6
0,lableled_yolot_dublin_day,0.0,0.0,0.0,0.0,0.22163541629745154,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.3333333333333333


In [33]:
s = sa.append(sb)
s

,df_name,accuracy,f1_score,precision_score,recall_score,r2_score,accuracy_with_range,f1_with_range,precision_with_range,recall_with_range,accuracy_with_bool
0,lableled_resnet_dublin_night,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.06666666666666667,0.6969771806776647,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.4666666666666667,0.6
0,lableled_yolov3_dublin_night,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,-0.398794823668873,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.2
0,lableled_yolot_dublin_night,0.0,0.0,0.0,0.0,-1.5167934406796406,0.4,0.4000000000000001,0.4,0.4,0.06666666666666667
0,lableled_resnet_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.8567149895438273,1.0,1.0,1.0,1.0,0.6666666666666666
0,lableled_yolov3_dublin_day,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.26666666666666666,0.47231772587105236,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.9333333333333333,0.6
0,lableled_yolot_dublin_day,0.0,0.0,0.0,0.0,0.22163541629745154,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.7333333333333333,0.3333333333333333


In [34]:
s.to_csv(path_or_buf=os.path.join(os.getcwd(), "algos_dublin_scores.csv"))